# 2. 短命作品の学習と予測

　目次情報を学習して，短命作品を予測してみます．

## 環境構築

```bash
conda env create -f env.yml
```

## 問題設定

　本記事の目的は，2週目から7週目までの掲載順を入力とし，当該作品が短命作品（10週以内に終了する作品）か否かを予測することです．1週目の掲載順を除外したのは，新連載作品はほとんど巻頭に掲載されるためです．また，近年最短とされる作品が8週連載なので，遅くともこの1週間前には打ち切りを予測するため，7週目までの掲載順を入力としました．打ち切りを阻止するためには，もっと早期に打ち切りを予測する必要がありますが，これは今後の課題ということで…．

## 目次データ

　[0_obtain_comic_data_j.ipynb](0_obtain_comic_data_j.ipynb)で取得した`data/wj-api.json`を使います．また，[1_analyze_comic_data_j.ipynb](1_analyze_comic_data_j.ipynb)で定義した`ComicAnalyzer`を`analyze.py`からimportして使います．

In [1]:
import analize

wj = analize.ComicAnalyzer()

## モデル

　以下に，本記事で想定する[多層パーセプトロン](https://en.wikipedia.org/wiki/Multilayer_perceptron)のモデルを示します．多層パーセプトロンについては，[誤差逆伝播法のノート](http://qiita.com/Ugo-Nama/items/04814a13c9ea84978a4c)が詳しいです．

![model.png](fig/model.png)

　本記事では，隠れ層1層の多層パーセプトロンを用います．入力層は6ノード（2週目から7週目までの掲載順），隠れ層は$n$ノード，出力層は1ノード（短命作品である確率）を含みます．隠れ層および出力層の活性化関数として，[Sigmoid関数](https://ja.wikipedia.org/wiki/%E6%B4%BB%E6%80%A7%E5%8C%96%E9%96%A2%E6%95%B0#.E3.82.B7.E3.82.B0.E3.83.A2.E3.82.A4.E3.83.89.E9.96.A2.E6.95.B0)を用います．学習率として，$r$を用います．最適化アルゴリズムとして，オーソドックスな[Stochastic Gradient Descent (SGD)](https://ja.wikipedia.org/wiki/%E7%A2%BA%E7%8E%87%E7%9A%84%E5%8B%BE%E9%85%8D%E9%99%8D%E4%B8%8B%E6%B3%95#cite_note-17)と，人気の[Adam](https://arxiv.org/abs/1412.6980)の両方を使って結果を比較します．
 
　本記事では，以下のハイパーパラメータを調整して，予測性能の最大化を目指します．

|項目|設計空間|補足|
|:--|:--|:--|
|学習率（$r$）| $0 < r < 1$| 大きいほど収束が速いが，大きすぎるとうまく学習しない．|
|隠れ層のノード数（$n$）| $\{1, 2,...,6\}$ |最大でも，入力層のノード数程度？|
|最適化アルゴリズム| {Gradient descent, Adam}| 前者はオーソドックス，Adamは新しめ．|


## 学習データ，テストデータの生成

In [2]:
import numpy as np

def make_x(anlzr, title='ONE PIECE'):
    worsts = np.array(anlzr.extract_item(title)[1:7])
    bests = np.array(anlzr.extract_item(title, 'best')[1:7])
    worsts_normalized = worsts / (worsts + bests - 1)    
    return worsts_normalized


def make_y(anlzr, title='ONE PIECE', thresh_week=10):
    return float(len(anlzr.extract_item(title)) <=  thresh_week)


def batch_x_y(anlzr, titles, thresh_week=10):
    xs = np.array([make_x(anlzr, title) for title in titles])
    ys = np.array([[make_y(anlzr, title, thresh_week)] for title in titles])
    return xs, ys

In [3]:
x_test_10, y_test_10 = batch_x_y(wj, wj.end_titles[-100:])
x_train_10, y_train_10 = batch_x_y(wj, wj.end_titles[:-100])

## 多層パーセプトロンの構築

　`ComicNet()`は，多層パーセプトロンを管理するクラスです．

In [4]:
import tensorflow as tf

class ComicNet():
    def __init__(self, x_train, y_train, x_test, y_test,
                 r=0.01, n_x=6, n_h=4, epoch=1000, algo='SGD', log=False):
        self.x_train = x_train[:, :n_x]
        self.y_train = y_train
        self.x_test = x_test[:, :n_x]
        self.y_test = y_test
        self.learning_rate = r
        self.n_x = n_x
        self.n_h = n_h
        self.epoch = epoch
        self.algo = algo
        self.log = log
                
    def build_network(self):
        tf.reset_default_graph()
        
        n_y = self.y_test.shape[1]
        self.x = tf.placeholder(tf.float32, [None, self.n_x], name='x')
        self.y = tf.placeholder(tf.float32, [None, n_y], name='y')
        
        self.w_h = tf.Variable(
            tf.truncated_normal((self.n_x, self.n_h), stddev=0.05))
        self.b_h = tf.Variable(tf.zeros(self.n_h))
        self.logits = tf.add(tf.matmul(self.x, self.w_h), self.b_h)
        self.logits = tf.nn.sigmoid(self.logits)
        tf.summary.histogram('w_h', self.w_h)
        tf.summary.histogram('b_h', self.b_h)
        
        self.w_y = tf.Variable(
            tf.truncated_normal((self.n_h, n_y), stddev=0.05))
        self.b_y = tf.Variable(tf.zeros(n_y))
        self.logits = tf.add(tf.matmul(self.logits, self.w_y), self.b_y)
        self.logits = tf.nn.sigmoid(self.logits, name='logits')
        tf.summary.histogram('w_b', self.w_y)
        tf.summary.histogram('b_b', self.b_y)
        tf.summary.histogram('logits', self.logits)
        
        self.loss = tf.nn.l2_loss(self.y - self.logits, name='loss')
        tf.summary.scalar('loss', self.loss)
        
        if self.algo == 'SGD':
            self.optimizer = tf.train.GradientDescentOptimizer(
                self.learning_rate).minimize(self.loss)
        else:
            self.optimizer = tf.train.AdamOptimizer(
                self.learning_rate).minimize(self.loss)
        
        correct_prediction = tf.equal(self.y, tf.round(self.logits))
        self.acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),
            name='acc')
        tf.summary.scalar('acc', self.acc)
        
        self.merged = tf.summary.merge_all()
    
    
    def batch_train_and_valid(self, n_tra, n_val):
        i_pos = [i for i in range(len(self.y_train))
                 if self.y_train[i]==1]
        i_neg = [i for i in range(len(self.y_train))
                 if self.y_train[i]==0]
            
        i_pos_tra = np.random.choice(i_pos, n_tra, replace=False)
        i_neg_tra = np.random.choice(i_neg, n_tra, replace=False)
        i_pos_val = np.random.choice(
            [i for i in i_pos if not i in i_pos_tra], n_val,
            replace=False)
        i_neg_val = np.random.choice(
            [i for i in i_neg if not i in i_neg_tra], n_val,
            replace=False)
        
        x_tra = self.x_train[np.r_[i_pos_tra, i_neg_tra]]
        y_tra = self.y_train[np.r_[i_pos_tra, i_neg_tra]]
        x_val = self.x_train[np.r_[i_pos_val, i_neg_val]]
        y_val = self.y_train[np.r_[i_pos_val, i_neg_val]]
        
        return x_tra, y_tra, x_val, y_val
        
        
    def train(self):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            log_tra = 'logs/n_x={}/tra/r={},n_h={},algo={}'\
                .format(self.n_x, self.learning_rate, self.n_h, self.algo)
            writer_tra = tf.summary.FileWriter(log_tra)
            log_val = 'logs/n_x={}/val/r={},n_h={},algo={}'\
                .format(self.n_x, self.learning_rate, self.n_h, self.algo)
            writer_val = tf.summary.FileWriter(log_val)

            for e in range(self.epoch):
                n_tra = 50
                n_val = (self.y_train == 1).sum() - n_tra
                x_tra, y_tra, x_val, y_val = self.batch_train_and_valid(n_tra, n_val)
                
                feed_dict = {self.x: x_tra, self.y: y_tra}
                _, loss_tra, acc_tra, mer_tra = sess.run(
                        (self.optimizer, self.loss, self.acc, self.merged), 
                        feed_dict=feed_dict)
                
                feed_dict = {self.x: x_val, self.y: y_val}
                loss_val, acc_val, mer_val = sess.run(
                    (self.loss, self.acc, self.merged),
                    feed_dict=feed_dict)
                
                if self.log:
                    writer_tra.add_summary(mer_tra, e)
                    writer_val.add_summary(mer_val, e)
            
            self.save_model_path = './prediction_model'
            saver = tf.train.Saver()
            save_path = saver.save(sess, self.save_model_path)
            
            
    def test(self):
        loaded_graph = tf.Graph()
        
        with tf.Session(graph=loaded_graph) as sess:
            loader = tf.train.import_meta_graph(
                self.save_model_path + '.meta')
            loader.restore(sess, self.save_model_path)
            
            x_loaded = loaded_graph.get_tensor_by_name('x:0')
            y_loaded = loaded_graph.get_tensor_by_name('y:0')
            
            loss_loaded = loaded_graph.get_tensor_by_name('loss:0')
            acc_loaded = loaded_graph.get_tensor_by_name('acc:0')
            logits_loaded = loaded_graph.get_tensor_by_name('logits:0')
        
            feed_dict = {x_loaded: self.x_test, y_loaded: self.y_test}
            loss_test, acc_test, logits_test = sess.run(
                (loss_loaded, acc_loaded, logits_loaded), feed_dict=feed_dict)
            print('acc_test:', acc_test)
            return logits_test

## ハイパーパラメータの調整

　以下では，[TensorBoard](https://www.tensorflow.org/get_started/summaries_and_tensorboard)を使ってハイパーパラメータを調整します．本記事におけるハイパーパラメータは，学習率$r$，隠れ層のノード数$n$，そして最適化アルゴリズム（SGD，Adam）です．まずは，アタリをつけるため，ざっくりとパラメータを振って検証データに対する精度を比較します．

In [ ]:
for n_x in range(1, 7):
    for r in [0.1, 0.05, 0.001, 0.005]:
        for n_h in range(1, n_x + 1):
            for algo in ['SGD', 'Adam']:
                print('n_x={}, r={}, n_h={}, algo={}'.
                      format(str(n_x), str(r), str(n_h), algo))
                wjnet = ComicNet(x_train_10, y_train_10, x_test_10, y_test_10, 
                                 n_x=n_x, r=r, n_h=n_h, algo=algo, epoch=10000, log=True)
                wjnet.build_network()
                wjnet.train()

n_x=1, r=0.1, n_h=1, algo=SGD
n_x=1, r=0.1, n_h=1, algo=Adam
n_x=1, r=0.05, n_h=1, algo=SGD


## 追加実験1. 20週以内に終了する作品の識別

　多くの方にご指摘された通り，10週以内に終了する作品は凄く稀です．以下は，歴代ジャンプ作品の掲載期間のヒストグラムです．
 
　なお，`thresh_week`を調整すれば，任意の期間内に終了する作品の識別問題に挑戦することができます．ご興味のある方はぜひ．

## 追加実験2. 連載期間の回帰

　短命作品か否かの識別問題だけではなく，連載期間の回帰問題にも挑戦します．多層パーセプトロンは以下です．短命作品の識別問題との違いは，出力層の活性化関数が無くなっていることです．